In [14]:
print("Available layouts:", layouts)

Available layouts: ['asymmetric_advantages', 'asymmetric_advantages_tomato', 'bonus_order_test', 'bottleneck', 'centre_objects', 'centre_pots', 'coordination_ring', 'corridor', 'counter_circuit', 'counter_circuit_o_1order', 'cramped_corridor', 'cramped_room', 'cramped_room_o_3orders', 'cramped_room_single', 'cramped_room_tomato', 'five_by_five', 'forced_coordination', 'forced_coordination_tomato', 'inverse_marshmallow_experiment', 'large_room', 'long_cook_time', 'm_shaped_s', 'marshmallow_experiment', 'marshmallow_experiment_coordination', 'mdp_test', 'multiplayer_schelling', 'old_dynamics_cook_test', 'old_dynamics_put_test', 'pipeline', 'random0', 'random3', 'scenario1_s', 'scenario2', 'scenario2_s', 'scenario3', 'scenario4', 'schelling', 'schelling_s', 'simple_o', 'simple_o_t', 'simple_tomato', 'small_corridor', 'soup_coordination', 'tutorial_0', 'tutorial_1', 'tutorial_2', 'tutorial_3', 'unident', 'you_shall_not_pass']


In [1]:
import sys
sys.path.append(".")

from stable_baselines3 import PPO
from env.overcooked_wrapper import OvercookedGym

c:\Users\Bushra\AppData\Local\Programs\Python\Python310\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
env = OvercookedGym("cramped_room")
print("Observation shape:", env.observation_space.shape)
print("Action space:", env.action_space)

Computing MediumLevelActionManager
Observation shape: (96,)
Action space: Discrete(6)


In [3]:
model = PPO(
    "MlpPolicy",
    env,
    learning_rate=3e-4,
    batch_size=64,
    n_steps=2048,
    verbose=1
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [4]:
model.learn(total_timesteps=200_000)

Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 400      |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 682      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 400        |
|    ep_rew_mean          | 0          |
| time/                   |            |
|    fps                  | 575        |
|    iterations           | 2

In [5]:
model.save("ppo_student")

In [13]:
import os

layout_dir = r"C:\Users\Bushra\BDML1\S9\overcooked_ai\src\overcooked_ai_py\data\layouts"

layouts = sorted([
    f.replace(".layout", "")
    for f in os.listdir(layout_dir)
    if f.endswith(".layout")
])

print("Nombre de layouts disponibles :", len(layouts))
print("---- Layouts ----")
for name in layouts:
    print("-", name)


Nombre de layouts disponibles : 49
---- Layouts ----
- asymmetric_advantages
- asymmetric_advantages_tomato
- bonus_order_test
- bottleneck
- centre_objects
- centre_pots
- coordination_ring
- corridor
- counter_circuit
- counter_circuit_o_1order
- cramped_corridor
- cramped_room
- cramped_room_o_3orders
- cramped_room_single
- cramped_room_tomato
- five_by_five
- forced_coordination
- forced_coordination_tomato
- inverse_marshmallow_experiment
- large_room
- long_cook_time
- m_shaped_s
- marshmallow_experiment
- marshmallow_experiment_coordination
- mdp_test
- multiplayer_schelling
- old_dynamics_cook_test
- old_dynamics_put_test
- pipeline
- random0
- random3
- scenario1_s
- scenario2
- scenario2_s
- scenario3
- scenario4
- schelling
- schelling_s
- simple_o
- simple_o_t
- simple_tomato
- small_corridor
- soup_coordination
- tutorial_0
- tutorial_1
- tutorial_2
- tutorial_3
- unident
- you_shall_not_pass


In [7]:
from overcooked_ai_py.mdp.overcooked_mdp import OvercookedGridworld

def print_layout(layout_name="cramped_room"):
    mdp = OvercookedGridworld.from_layout_name(layout_name)
    grid = mdp.terrain_mtx
    
    print(f"Layout: {layout_name}\n")
    for row in grid:
        print("".join(row))

print_layout("cramped_room")

Layout: cramped_room

XXPXX
O   O
X   X
XDXSX


- X : Wall (blocked)
- P : Player Spawn
- O : Onion
- D : Dish
- S : pot

In [15]:
from stable_baselines3 import PPO
from env.overcooked_wrapper import OvercookedGym

env = OvercookedGym("cramped_room")
model = PPO.load("ppo_student", env=env)

obs, _ = env.reset()
for i in range(10):
    action, _ = model.predict(obs)
    obs, reward, done, trunc, info = env.step(action)
    print(action, reward)

Computing MediumLevelActionManager
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Computing MediumLevelActionManager
1 0
1 0
4 0
1 0
1 0
4 0
1 0
1 0
1 0
1 0


In [2]:
from stable_baselines3 import PPO
from env.overcooked_wrapper import OvercookedGym
from overcooked_ai_py.agents.agent import Agent
import numpy as np


In [23]:
from overcooked_ai_py.mdp.actions import Action

class PPOStudentAgent:
    """
    Overcooked-compatible agent that wraps a Stable-Baselines3 PPO model.
    """

    def __init__(self, model, featurize_fn):
        self.model = model
        self.featurize_fn = featurize_fn
        self.agent_index = None
        self.mdp = None

    def reset(self):
        """Required by Overcooked Agent API."""
        pass

    def set_agent_index(self, idx):
        """Overcooked gives agent 0 or 1."""
        self.agent_index = idx

    def set_mdp(self, mdp):
        """Required by AgentEvaluator → store reference to MDP."""
        self.mdp = mdp

    def action(self, state):
        """
        Called at every step by Overcooked.
        Must return:
            (overcooked_action, info_dict)
        """

        # 1. Featurize state
        obs0, obs1 = self.featurize_fn(state)
        obs = obs0 if self.agent_index == 0 else obs1

        # 2. PPO predicts discrete action index
        action_idx, _ = self.model.predict(obs, deterministic=True)

        # 3. Convert to Overcooked action
        oc_action = Action.INDEX_TO_ACTION[action_idx]

        return oc_action, {}


In [19]:
env = OvercookedGym("cramped_room")
model = PPO.load("ppo_student", env=env)

Computing MediumLevelActionManager
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## intialize agent

In [24]:
featurize_fn = env.env.featurize_state_mdp

agent0 = PPOStudentAgent(model, featurize_fn)
agent1 = PPOStudentAgent(model, featurize_fn)

pair = AgentPair(agent0, agent1)


In [ ]:
trajs = ae.evaluate_agent_pair(pair, num_games=1)

from overcooked_ai_py.visualization.state_visualizer import StateVisualizer
StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)


Avg rew: 0.00 (std: 0.00, se: 0.00); avg len: 400.00; : 100%|██████████| 1/1 [00:00<00:00,  2.63it/s]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…